In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import networkx as nx

# Function to visualize group relationships as a network graph
def visualize_group_network(results):
    G = nx.Graph()
    
    # Add nodes (people)
    for i in range(len(results['faces'])):
        emotion = results['emotions'][i]['emotion']
        
        # Find group
        group_id = None
        for g_id, members in results['groups'].items():
            if i in members:
                group_id = g_id
                break
        
        G.add_node(i, emotion=emotion, group=group_id)
    
    # Add edges based on proximity and gaze
    for i in range(len(results['faces'])):
        # Add gaze edges
        for gaze in results['gazes']:
            if gaze['source_idx'] == i and gaze['target_idx'] is not None:
                G.add_edge(i, gaze['target_idx'], type='gaze', weight=2)
        
        # Add proximity edges for people in same group
        for j in range(i+1, len(results['faces'])):
            # Check if in same group
            same_group = False
            for members in results['groups'].values():
                if i in members and j in members:
                    same_group = True
                    break
                    
            if same_group:
                G.add_edge(i, j, type='proximity', weight=1)
    
    # Define node colors based on emotions
    emotion_colors = {
        'Angry': 'red',
        'Disgust': 'purple',
        'Fear': 'black',
        'Happy': 'green',
        'Sad': 'blue',
        'Surprise': 'orange',
        'Neutral': 'gray'
    }
    
    node_colors = [emotion_colors.get(G.nodes[n]['emotion'], 'gray') for n in G.nodes]
    
    # Define edge styles
    edge_styles = []
    edge_weights = []
    
    for u, v, data in G.edges(data=True):
        if data['type'] == 'gaze':
            edge_styles.append('dashed')
            edge_weights.append(2)
        else:
            edge_styles.append('solid')
            edge_weights.append(1)
    
    # Create the plot
    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G)
    
    # Draw the graph
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=500)
    
    # Draw edges with different styles
    for (u, v, data), style, width in zip(G.edges(data=True), edge_styles, edge_weights):
        nx.draw_networkx_edges(
            G, pos, edgelist=[(u, v)],
            width=width,
            style=style,
            alpha=0.7
        )
    
    # Add labels
    labels = {n: f"{n}: {G.nodes[n]['emotion']}" for n in G.nodes}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=10)
    
    # Add a legend for emotions
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                                 markerfacecolor=color, markersize=10, label=emotion)
                      for emotion, color in emotion_colors.items()]
    
    plt.legend(handles=legend_elements, title="Emotions")
    plt.title("Group Dynamics Network Analysis")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Function to create a heatmap of group emotions
def visualize_group_emotions(results):
    # Extract group emotions
    groups = {}
    for group_id, analysis in results['group_analysis']['group_emotions'].items():
        emotion_counts = analysis['emotion_counts']
        total = sum(emotion_counts.values())
        
        # Convert to percentages
        emotion_percentages = {e: (c/total)*100 for e, c in emotion_counts.items()}
        groups[group_id] = emotion_percentages
    
    # Create a complete set of emotions
    all_emotions = set()
    for emotions in groups.values():
        all_emotions.update(emotions.keys())
    all_emotions = sorted(list(all_emotions))
    
    # Create the matrix for the heatmap
    matrix = []
    group_ids = sorted(groups.keys())
    
    for group_id in group_ids:
        row = []
        for emotion in all_emotions:
            row.append(groups[group_id].get(emotion, 0))
        matrix.append(row)
    
    # Create the heatmap
    plt.figure(figsize=(10, 6))
    plt.imshow(matrix, cmap='YlOrRd')
    
    # Add labels
    plt.yticks(range(len(group_ids)), [f"Group {g}" for g in group_ids])
    plt.xticks(range(len(all_emotions)), all_emotions, rotation=45)
    
    # Add colorbar and title
    plt.colorbar(label='Percentage')
    plt.title('Emotion Distribution by Group')
    plt.tight_layout()
    plt.show()

# Example usage
# visualize_group_network(results)
# visualize_group_emotions(results)